# Two-layer hybrid recommender system in retail

**Data:** from [Retail X5 Hero Competition](https://retailhero.ai/c/recommender_system/overview)

**Stack:**

- 1-st layer: NLP, Implicit, ItemItemRecommender, ALS, sklearn, pandas, numpy, matplotlib
- 2-nd layer: CatBoost, LightGBM



**Stages**:

1. Prepare data:  prefiltering 
2. Learn 1-st layer model (this is baseline based on MainRecommender class)
3. Choose Top popular
4. Choose Top@k Recall
5. Add feature engineering for ranking
6. Learn 2-nd layer model
7. Validate on test.csv

## Prepare data